# Conexión a Bases de datos

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


- Abrir Anaconda prompt
- Instalar el conector de MySQL para python

    `conda install -c anaconda mysql-connector-python`

In [47]:
import numpy as np
import pandas as pd
import mysql.connector as sql

In [48]:
# Database args
db_user = 'intensivo@banistmoserver'
db_passwd = '1234.intenso'
db_host = 'banistmoserver.mysql.database.azure.com'
db_name = 'intensivopython'

In [63]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

Usando el cursor de SQL. Ojo que antes de cerrar el cursor tenemos que realizar las operaciones de lectura que necesitemos.

In [50]:
query= "SHOW TABLES;"

cursor = cnx.cursor(buffered=True)
cursor.execute(query)

for el in cursor:
    print(el)

cursor.close()

('comunas',)
('provincias',)
('regiones',)


True

Usando el método read_sql de Pandas

In [51]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

,Tables_in_intensivopython
0,comunas
1,provincias
2,regiones


Hagamos un select en la tabla regiones

In [52]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,1,Arica y Parinacota,XV
1,2,Tarapacá,I
2,3,Antofagasta,II
3,4,Atacama,III
4,5,Coquimbo,IV
5,6,Valparaíso,V
6,7,Metropolitana de Santiago,RM
7,8,Libertador General Bernardo O'Higgins,VI
8,9,Maule,VII
9,10,Biobío,VIII


La siguiente consulta está parametrizada. Fíjese cómo se declaran los parámetros dentro de la cadena de la consulta, y cómo después se ingresan, en el comando read_sql de Pandas.

In [53]:
query= "SELECT * from regiones WHERE region_id BETWEEN %s and %s;"
regiones=pd.read_sql(query, params=(2,8), con=cnx)
regiones.head()

,region_id,region_nombre,region_ordinal
0,2,Tarapacá,I
1,3,Antofagasta,II
2,4,Atacama,III
3,5,Coquimbo,IV
4,6,Valparaíso,V


Como verá en la siguiente consulta, estamos utilizando cosas aprendidas en los tutoriales anteriores. Estamos armando dinámicamente la cadena que respresenta una consulta. Específicamente, utilizamos una lista con id's de regiones que nos interesan, lo convertimos a un objeto `array` de numpy (np.array), después aplicamos la función `map` para aplicar la función `str` al array, y finalmente, concatenamos los resultados con el caracter ",". **Todo en una línea**.
¿En cuantas líneas podría hacer esto en Java?

In [54]:
','.join(["Hola","Mundo","Como","Estan"])

'Hola,Mundo,Como,Estan'

In [55]:
region_ids=[5,8,10]

query= "SELECT * from regiones WHERE region_id IN ("
query_region= ', '.join(map(str, np.array(region_ids)))
query= query + query_region + ");"
print(query)

regiones=pd.read_sql(query, con=cnx)
regiones

SELECT * from regiones WHERE region_id IN (5, 8, 10);


,region_id,region_nombre,region_ordinal
0,5,Coquimbo,IV
1,8,Libertador General Bernardo O'Higgins,VI
2,10,Biobío,VIII


## Uso de pandas.merge()

In [56]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,1,Arica y Parinacota,XV
1,2,Tarapacá,I
2,3,Antofagasta,II
3,4,Atacama,III
4,5,Coquimbo,IV
5,6,Valparaíso,V
6,7,Metropolitana de Santiago,RM
7,8,Libertador General Bernardo O'Higgins,VI
8,9,Maule,VII
9,10,Biobío,VIII


In [57]:
query= "SELECT * from provincias;"
provincias=pd.read_sql(query, con=cnx)
provincias.head()

,provincia_id,provincia_nombre,region_id
0,1,Arica,1
1,2,Parinacota,1
2,3,Iquique,2
3,4,El Tamarugal,2
4,5,Antofagasta,3


In [58]:
query= "SELECT * from comunas;"
comunas=pd.read_sql(query, con=cnx)
comunas.head()

,comuna_id,comuna_nombre,provincia_id
0,1,Arica,1
1,2,Camarones,1
2,3,General Lagos,2
3,4,Putre,2
4,5,Alto Hospicio,3


In [59]:
regiones_provincias=pd.merge(regiones, provincias, how='outer')
regiones_provincias.head()

,region_id,region_nombre,region_ordinal,provincia_id,provincia_nombre
0,1,Arica y Parinacota,XV,1,Arica
1,1,Arica y Parinacota,XV,2,Parinacota
2,2,Tarapacá,I,3,Iquique
3,2,Tarapacá,I,4,El Tamarugal
4,3,Antofagasta,II,5,Antofagasta


## Análisis de la población por comuna

In [65]:
query= "SELECT * from poblacion;"
poblacion=pd.read_sql(query, con=cnx)
poblacion.head()

,comuna_nombre,region_nombre,provincia_nombre,poblacion_1992,poblacion_2002,poblacion_2017
0,Antofagasta,Antofagasta,Antofagasta,227065.0,289477.0,361873
1,Calama,Antofagasta,El Loa,121827.0,137144.0,165731
2,Tocopilla,Antofagasta,Tocopilla,25119.0,23968.0,25186
3,Mejillones,Antofagasta,Antofagasta,5886.0,8034.0,13467
4,Taltal,Antofagasta,Antofagasta,10926.0,10307.0,13317


In [66]:
(poblacion.groupby(poblacion['region_nombre'])['poblacion_1992','poblacion_2002','poblacion_2017'].sum())

,poblacion_1992,poblacion_2002,poblacion_2017
region_nombre,,,
Antofagasta,408874.0,481931.0,607534
Arica y Parinacota,172669.0,188463.0,226068
Atacama,229154.0,253205.0,286168
Aysén del General Carlos Ibáñez del Campo,78666.0,89986.0,103158
Biobío,1729209.0,1859546.0,2037414
Coquimbo,501795.0,603133.0,757586
La Araucanía,777788.0,867351.0,957224
Libertador General Bernardo O'Higgins,690751.0,775883.0,914555
Los Lagos,658199.0,712039.0,828708


In [69]:
regiones_poblacion=pd.merge(regiones, poblacion, how='outer')
regiones_poblacion

,region_id,region_nombre,region_ordinal,comuna_nombre,provincia_nombre,poblacion_1992,poblacion_2002,poblacion_2017
0,1,Arica y Parinacota,XV,Arica,Arica,168633.0,184914.0,221364
1,1,Arica y Parinacota,XV,Putre,Parinacota,2417.0,1671.0,2765
2,1,Arica y Parinacota,XV,Camarones,Arica,729.0,999.0,1255
3,1,Arica y Parinacota,XV,General Lagos,Parinacota,890.0,879.0,684
4,2,Tarapacá,I,Iquique,Iquique,144447.0,168397.0,191468
5,2,Tarapacá,I,Alto Hospicio,Iquique,5511.0,49436.0,108375
6,2,Tarapacá,I,Pozo Almonte,Tamarugal,6209.0,9407.0,15711
7,2,Tarapacá,I,Pica,Tamarugal,2332.0,3498.0,9296
8,2,Tarapacá,I,Huara,Tamarugal,1931.0,2522.0,2730
9,2,Tarapacá,I,Colchane,Tamarugal,1554.0,1474.0,1728


In [70]:
(poblacion.groupby(poblacion['region_nombre'])['poblacion_1992','poblacion_2002','poblacion_2017'].sum())

,poblacion_1992,poblacion_2002,poblacion_2017
region_nombre,,,
Antofagasta,408874.0,481931.0,607534
Arica y Parinacota,172669.0,188463.0,226068
Atacama,229154.0,253205.0,286168
Aysén del General Carlos Ibáñez del Campo,78666.0,89986.0,103158
Biobío,1729209.0,1859546.0,2037414
Coquimbo,501795.0,603133.0,757586
La Araucanía,777788.0,867351.0,957224
Libertador General Bernardo O'Higgins,690751.0,775883.0,914555
Los Lagos,658199.0,712039.0,828708


Cerramos la conexión al servidor de MySQL. **¡¡No olvide hacer esto!!**

In [ ]:
cnx.close()